In [1]:
import numpy as np
from datetime import datetime

np.random.seed(0)

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
class Diabetes:
    def __init__(self, i_node, h_node, o_node, lr):
        self.learning_rate = lr
        
        self.W2 = np.random.rand(i_node, h_node)
        self.b2 = np.random.rand(1)
        
        self.W3 = np.random.rand(h_node, o_node)
        self.b3 = np.random.rand(1)
        
        print("========== Diabetes Object is created. ==========\n")
        
    def feed_forward(self, x_data, t_data):
        delta = 1e-7
        
        z2 = np.dot(self.x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((self.t_data*np.log10(a3 + delta)) + ((1 - self.t_data)*np.log10((1 - a3) + delta)))

    def loss_val(self):
        delta = 1e-7
        
        z2 = np.dot(self.x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((self.t_data*np.log10(a3 + delta)) + ((1 - self.t_data)*np.log10((1 - a3) + delta)))
    
    def train(self, x_data, t_data):
        self.x_data = x_data
        self.t_data = t_data
        
        f = lambda x : self.feed_forward(self.x_data, self.t_data)
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
    def predict(self, test_data):
        z2 = np.dot(test_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        if a3 >= 0.5:
            result = 1
        else:
            result = 0
    
        return a3, result
    
    def accuracy(self, test_xdata, test_tdata):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(test_xdata)
        
        return accuracy_val

In [3]:
class DataGeneration:
    def __init__(self, file_name, split_rate):
        try:
            self.data = np.loadtxt(file_name, delimiter=',', dtype=np.float32)
        except Exception as err:
            print('[DataGeneration::generate()]  ', str(err))
            raise Exception(str(err))
            
        self.split_rate = split_rate
        
    def generate(self):
        np.random.shuffle(self.data)
        
        training_data = self.data[ :int(len(self.data)*self.split_rate)]
        test_data = self.data[int(len(self.data)*self.split_rate): ]
    
        print("training_data.shape = ", training_data.shape, ", test_data.shape = ", test_data.shape)
        
        return training_data, test_data

In [5]:
try:
    data_obj = DataGeneration('./diabetes.csv', 0.7)

    (training_data, test_data) = data_obj.generate()
    
except Exception as err:
    print('Exception Occur !!')
    print(str(err))

training_data.shape =  (531, 9) , test_data.shape =  (228, 9)


In [8]:
i_node = 8
h_node = 10
o_node = 1
lr = 1e-1
epoch = 20

obj1 = Diabetes(i_node, h_node, o_node, lr)

print("Neural Network Learning using Numerical Derivative...\n")

start_time = datetime.now()

for step in range(epoch):
    for index in range(len(training_data)):
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj1.train(input_data, target_data)
        
    if step % (int)(0.05*epoch) == 0:
        print("epochs = ", step, " : ", "loss value = ", obj1.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

========== Diabetes Object is created. ==========

Neural Network Learning using Numerical Derivative...

epochs =  0  :  loss value =  0.17364760967075055
epochs =  1  :  loss value =  0.16986095605203097
epochs =  2  :  loss value =  0.157731655685238
epochs =  3  :  loss value =  0.14242548097932295
epochs =  4  :  loss value =  0.12806862228076069
epochs =  5  :  loss value =  0.11600884998304231
epochs =  6  :  loss value =  0.10634683657520823
epochs =  7  :  loss value =  0.09878485608067498
epochs =  8  :  loss value =  0.09293429298244599
epochs =  9  :  loss value =  0.08842479155998442
epochs =  10  :  loss value =  0.08494132365817265
epochs =  11  :  loss value =  0.08223142908653516
epochs =  12  :  loss value =  0.08010011155866963
epochs =  13  :  loss value =  0.0784005179272407
epochs =  14  :  loss value =  0.07702403441250509
epochs =  15  :  loss value =  0.07589130118675153
epochs =  16  :  loss value =  0.07494469381178286
epochs =  17  :  loss value =  0.0741423

In [9]:
try:
    input_data = test_data[:, 0:-1]
    target_data = test_data[:, [-1]]

    accuracy_ret = obj1.accuracy(input_data, target_data)
    
    print('Accuracy => ', accuracy_ret)
    
except Exception as err:
    print(str(err))

Accuracy =>  0.7456140350877193
